In [41]:
using Pkg
Pkg.instantiate()
using TeneT

# 2D classical Ising model

## Step 1: define M tensor
 <img src="../picture/build_M_tensor.png" width = "30%" height = "30%" align=center />

In [53]:
using TeneT: _arraytype
using OMEinsum
using Zygote

const isingβc = log(1+sqrt(2))/2

abstract type HamiltonianModel end

struct Ising <: HamiltonianModel 
    Ni::Int
    Nj::Int
    β::Float64
end

"""
    model_tensor(model::Ising, type)
return the  `MT <: HamiltonianModel` `type` tensor at inverse temperature `β` for  two-dimensional
square lattice tensor-network.
"""
function model_tensor(model::Ising, ::Val{:bulk})
    Ni, Nj, β = model.Ni, model.Nj, model.β
    ham = Zygote.@ignore ComplexF64[-1. 1;1 -1]
    w = exp.(- β * ham)
    wsq = sqrt(w)
    m = ein"ia,ib,ic,id -> abcd"(wsq, wsq, wsq, wsq)

    M = Zygote.Buffer(m, 2,2,2,2,Ni,Nj)
    @inbounds @views for j = 1:Nj,i = 1:Ni
        M[:,:,:,:,i,j] = m
    end
    return copy(M)
end

function model_tensor(model::Ising, ::Val{:mag})
    Ni, Nj, β = model.Ni, model.Nj, model.β
    a = reshape(ComplexF64[1 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 -1], 2,2,2,2)
    cβ, sβ = sqrt(cosh(β)), sqrt(sinh(β))
    q = 1/sqrt(2) * [cβ+sβ cβ-sβ; cβ-sβ cβ+sβ]
    m = ein"abcd,ai,bj,ck,dl -> ijkl"(a,q,q,q,q)
    M = Zygote.Buffer(m, 2,2,2,2,Ni,Nj)
    @inbounds @views for j = 1:Nj,i = 1:Ni
        M[:,:,:,:,i,j] = m
    end
    return copy(M)
end

function model_tensor(model::Ising, ::Val{:energy})
    Ni, Nj, β = model.Ni, model.Nj, model.β
    ham = ComplexF64[-1 1;1 -1]
    w = exp.(-β .* ham)
    we = ham .* w
    wsq = sqrt(w)
    wsqi = wsq^(-1)
    e = (ein"ai,im,bm,cm,dm -> abcd"(wsqi,we,wsq,wsq,wsq) + ein"am,bi,im,cm,dm -> abcd"(wsq,wsqi,we,wsq,wsq) + 
        ein"am,bm,ci,im,dm -> abcd"(wsq,wsq,wsqi,we,wsq) + ein"am,bm,cm,di,im -> abcd"(wsq,wsq,wsq,wsqi,we)) / 2
    M = Zygote.Buffer(e, 2,2,2,2,Ni,Nj)
    @inbounds @views for j = 1:Nj,i = 1:Ni
        M[:,:,:,:,i,j] = e
    end
    return copy(M)
end

model_tensor (generic function with 3 methods)

## Step 2: use TeneT.obs_env to get environment

In [54]:
using Random
Random.seed!(100)

β = 0.5
model = Ising(1, 1, β)
M = model_tensor(model, Val(:bulk))
env = TeneT.obs_env(M; χ=10, maxiter=10, miniter=1, 
                    updown=false, verbose=true, show_every=1);

↑ random initial 1×1 vumps_χ10 environment-> vumps@step: 1, error=Inf


vumps@step: 2, error=0.01730159744333545
vumps@step: 3, error=0.0006110217932778949
vumps@step: 4, error=8.525419213658111e-7
vumps@step: 5, error=1.486693561895049e-7


vumps@step: 6, error=3.5210780596961775e-8
vumps@step: 7, error=7.535505333349743e-9
vumps@step: 8, error=1.640536608963289e-9
vumps@step: 9, error=3.7020363904278236e-10
vumps done@step: 9, error=8.508814802635002e-11


## Step 3: use env to calculate observable

In [51]:
using TeneT: ALCtoAC
"""
    observable(env, model::MT, type)
return the `type` observable of the `model`. Requires that `type` tensor defined in model_tensor(model, Val(:type)).
"""
function observable(env, model::MT, ::Val{:Z}) where {MT <: HamiltonianModel}
    _, ALu, Cu, ARu, ALd, Cd, ARd, FL, FR, FLu, FRu = env
    atype = _arraytype(ALu)
    M   = atype(model_tensor(model, Val(:bulk)))
    χ,D,Ni,Nj = size(ALu)[[1,2,4,5]]
    
    z_tol = 1
    ACu = ALCtoAC(ALu, Cu)

    for j = 1:Nj,i = 1:Ni
        ir = i + 1 - Ni * (i==Ni)
        jr = j + 1 - Nj * (j==Nj)
        z = ein"(((adf,abc),dgeb),ceh),fgh ->"(FLu[:,:,:,i,j],ACu[:,:,:,i,j],M[:,:,:,:,i,j],FRu[:,:,:,i,j],conj(ACu[:,:,:,ir,j]))
        λ = ein"(acd,ab),(bce,de) ->"(FLu[:,:,:,i,jr],Cu[:,:,i,j],FRu[:,:,:,i,j],conj(Cu[:,:,ir,j]))
        z_tol *= Array(z)[]/Array(λ)[]
    end
    return z_tol^(1/Ni/Nj)
end

function observable(env, model::MT, type) where {MT <: HamiltonianModel}
    _, ALu, Cu, ARu, ALd, Cd, ARd, FL, FR, FLu, FRu = env
    χ,D,Ni,Nj = size(ALu)[[1,2,4,5]]
    atype = _arraytype(ALu)
    M     = atype(model_tensor(model, Val(:bulk)))
    M_obs = atype(model_tensor(model, type      ))
    obs_tol = 0
    ACu = ALCtoAC(ALu, Cu)
    ACd = ALCtoAC(ALd, Cd)

    for j = 1:Nj,i = 1:Ni
        ir = Ni + 1 - i
        obs = ein"(((adf,abc),dgeb),fgh),ceh -> "(FL[:,:,:,i,j],ACu[:,:,:,i,j],M_obs[:,:,:,:,i,j],ACd[:,:,:,ir,j],FR[:,:,:,i,j])
        λ = ein"(((adf,abc),dgeb),fgh),ceh -> "(FL[:,:,:,i,j],ACu[:,:,:,i,j],M[:,:,:,:,i,j],ACd[:,:,:,ir,j],FR[:,:,:,i,j])
        obs_tol += Array(obs)[]/Array(λ)[]
    end
    if type == Val(:mag)
        obs_tol = abs(obs_tol)
    end
    return obs_tol/Ni/Nj
end

"""
    magofβ(::Ising,β)
return the analytical result for the magnetisation at inverse temperature
`β` for the 2d classical ising model.
"""
magofβ(model::Ising) = model.β > isingβc ? (1-sinh(2*model.β)^-4)^(1/8) : 0.

magofβ

In [58]:
using Test

@testset "$(Ni)x$(Nj) ising forward with $atype" for Ni = [1], Nj = [1], atype = [Array]
    Random.seed!(100)
    β = 0.5
    model = Ising(Ni, Nj, β)
    M = atype(model_tensor(model, Val(:bulk)))
    env = obs_env(M; χ = 10, maxiter = 10, miniter = 1, 
         infolder = "./example/data/$model/", 
        outfolder = "./example/data/$model/", 
        updown = false, verbose = false, savefile = false
        )
    @test observable(env, model, Val(:Z)     ) ≈ 2.789305993957602
    @test observable(env, model, Val(:mag)   ) ≈ magofβ(model) 
    @test observable(env, model, Val(:energy)) ≈ -1.745564581767667
end

Test Summary:                | Pass  Total
1x1 ising forward with Array |    3      3


1-element Vector{Any}:
 Test.DefaultTestSet("1x1 ising forward with Array", Any[], 3, false, false)

## Step 4(optional): calculate energy by Zygote.gradient

In [52]:
@testset "$(Ni)x$(Nj) ising backward with $atype" for Ni = [1], Nj = [1], atype = [Array]
    Random.seed!(100)
    function logZ(β)
        model = Ising(1, 1, β)
        M = model_tensor(model, Val(:bulk))
        env = obs_env(M;χ = 10, maxiter = 10, miniter = 1, 
                        updown = false, verbose = false, savefile = false
                    )
        log(real(observable(env, model, Val(:Z))))
    end
    @test Zygote.gradient(β->-logZ(β), 0.5)[1] ≈ -1.745564581767667
end

Test Summary:                 | Pass  Total
1x1 ising backward with Array |    1      1


1-element Vector{Any}:
 Test.DefaultTestSet("1x1 ising backward with Array", Any[], 1, false, false)

<font size=5>*try different Temperature $\beta$, bond dimension $\chi$, unit cell size $Ni \times Nj$ and CuArray with GPU!*</font>